In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [2]:
# # Restart kernel to clear memory
# !kill -9 -1


In [3]:
df = pd.read_csv('/kaggle/input/cleaned/preprocessed_bangla_news.csv', usecols=['clean_summary', 'clean_article'])

In [4]:
df.dropna()

,clean_article,clean_summary
0,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...
1,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ র ...,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ র ...
2,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...
3,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...
4,বিএনপির অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রতিব...,বিএনপির অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রতিব...
...,...,...
35111,রাশিয়ায় প্রেসিডেন্ট নির্বাচনের জন্য দেশব্যাপী ...,রাশিয়ায় প্রেসিডেন্ট নির্বাচনের জন্য দেশব্যাপী ...
35112,রমজান মাসে সিলেট অঞ্চলে পুড়ির মেয়ে বাড়ি ইফতারি...,রমজান মাসে সিলেট অঞ্চলে পুড়ির মেয়ে বাড়ি ইফতারি...
35113,বঙ্গবন্ধু শেখ মুজিবুর রহমান এমন একটি নামযে নাম...,বঙ্গবন্ধু শেখ মুজিবুর রহমান এমন একটি নামযে নাম...
35114,আমার ভাই মনে কষ্ট নিয়ে চলে গেছে সে মনে করত তাক...,আমার ভাই মনে কষ্ট নিয়ে চলে গেছে সে মনে করত তাক...


In [5]:
df = df.dropna(subset=['clean_article', 'clean_summary'])

In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)
df_val, df_test = train_test_split(df_test, test_size=0.65,shuffle=True)

In [7]:
# from datasets import Dataset
# ds_train = Dataset.from_pandas(df_train)
# ds_test = Dataset.from_pandas(df_test)

In [8]:
df_train['clean_summary'][50]

'বাবাকে হানানোর এক সপ্তাহ পরেই মারা গেলেন হলিউড অভিনেতা অ্যাঙ্গাস ক্লাউড মাত্র ২৫ বছর বয়সী এ অভিনেতার মৃত্যুর খবরটি নিশ্চিত করে তার ব্যক্তিগত ম্যানেজার ক্যাট বেইলি গণমাধ্যমকে জানিয়েছেন ক্লাউড সোমবার ৩১ জুলাই ক্যালিফোর্নিয়ার ওকল্যান্ডে নিজের বাড়িতে মারা গেছেন'

In [9]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os


# Specify the model name for mT5
model_name = "google/mt5-base"  # You can also use 'mt5-small', 'mt5-large', etc.

# Load the mT5 model
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Load the mT5 tokenizer
tokenizer = MT5Tokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
model_name

'google/mt5-base'

In [11]:
from torch.utils.data import Dataset, DataLoader

class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        # Drop rows with NaN values in article or summary columns
        data = data.dropna(subset=['clean_article', 'clean_summary'])

        # Tokenize input text and labels (without normalization)
        self.input_text = data['clean_article'].tolist()
        self.labels = data['clean_summary'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        # Tokenize the input and labels
        inputs = self.tokenizer(self.input_text[idx], 
                                max_length=self.max_length, 
                                padding='max_length', 
                                truncation=True, 
                                return_tensors="pt")

        labels = self.tokenizer(self.labels[idx], 
                                max_length=self.max_length, 
                                padding='max_length', 
                                truncation=True, 
                                return_tensors="pt")

        # Return input and labels for the sample
        return {
            'input_ids': inputs['input_ids'].squeeze(), 
            'attention_mask': inputs['attention_mask'].squeeze(), 
            'labels': labels['input_ids'].squeeze()
        }


In [12]:
import torch
from transformers import DataCollatorForSeq2Seq
from torch.nn.utils.rnn import pad_sequence

class MyDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __call__(self, features):
        batch = {}

        # Handle input_ids and attention_mask
        batch["input_ids"] = torch.stack([feature["input_ids"] for feature in features])
        batch["attention_mask"] = torch.stack([feature["attention_mask"] for feature in features])

        # Check if the labels are PyTorch tensors
        if isinstance(features[0]["labels"], torch.Tensor):
            # Ensure padding for labels is handled correctly
            batch["labels"] = torch.stack([feature["labels"] for feature in features])
        else:
            # Convert list of lists (if not tensor) to tensor and pad
            batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        # Make sure labels are padded if necessary (for variable-length sequences)
        if self.label_pad_token_id is not None:
            batch["labels"] = pad_sequence([feature["labels"] for feature in features], 
                                           batch_first=True, 
                                           padding_value=self.label_pad_token_id)

        return batch


In [13]:
# Create train, test, and validation datasets
train_dataset = Seq2SeqDataset(df_train, tokenizer)
test_dataset = Seq2SeqDataset(df_test, tokenizer)
validation_dataset = Seq2SeqDataset(df_val, tokenizer)

# Create DataLoader for each dataset
train_dataloader = DataLoader(train_dataset, 
                              batch_size=16, 
                              shuffle=True, 
                              collate_fn=MyDataCollatorForSeq2Seq(tokenizer))

test_dataloader = DataLoader(test_dataset, 
                             batch_size=16, 
                             collate_fn=MyDataCollatorForSeq2Seq(tokenizer))

validation_dataloader = DataLoader(validation_dataset, 
                                   batch_size=16, 
                                   collate_fn=MyDataCollatorForSeq2Seq(tokenizer))


In [14]:
# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
device

device(type='cuda')

In [16]:
# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    eps=1e-8,
    weight_decay=0.01,
)

In [17]:
# Define the TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_total_limit=1,
    save_steps=5000,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=False,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=200,
    
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# a7eadb1b11af8e041a6e1104b836b061cb0ff681


In [19]:
data_collator = MyDataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,  # Padding to the model's max length
    max_length=128,  # Use the same max length as during tokenization
    label_pad_token_id=tokenizer.pad_token_id,
)


In [20]:
from transformers.trainer import Trainer

class CustomTrainer(Trainer):
    def _save(self, output_dir=None, state_dict=None):
        # Ensure all tensors are contiguous before saving
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()
        
        # Continue with normal save operation
        super()._save(output_dir, state_dict)

# Use CustomTrainer instead of Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),
)


In [21]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,0.024200,0.015574
1,0.016600,0.011537
2,0.013600,0.010632
3,0.010800,0.010492
4,0.010500,0.010578
5,0.007800,0.011132
6,0.005700,0.010756
8,0.003500,0.012864
9,0.003000,0.013103


TrainOutput(global_step=7680, training_loss=0.10352895418570067, metrics={'train_runtime': 18589.3831, 'train_samples_per_second': 13.221, 'train_steps_per_second': 0.413, 'total_flos': 7.366130077099622e+16, 'train_loss': 0.10352895418570067, 'epoch': 9.998372660699756})

In [22]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5c2d7174480b9dc50c727c43a7c17498832b7f53d40248f237f3990d359cde4a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.3 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Evaluate ROUGE scores

In [25]:
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Smoothing function for BLEU
smoothing_function = SmoothingFunction().method4

# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch

# Initialize lists to store generated summaries and references
generated_summaries = []
references = []

# Generate summaries for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']
    labels = batch['labels']

    # Generate summaries
    with torch.no_grad():
        summary_ids = model.generate(
            input_text,
            num_beams=6,
            length_penalty=1.0,
            early_stopping=True
        )

    # Move the summary_ids to CPU to decode
    summary_ids = summary_ids.to('cpu')

    # Decode generated summaries and labels
    generated_summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    reference_summary = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Extend lists with generated summaries and references
    generated_summaries.extend(generated_summary)
    references.extend(reference_summary)

# Filter out empty predictions and references
filtered_generated_summaries = []
filtered_references = []

for pred, ref in zip(generated_summaries, references):
    if pred.strip() and ref.strip():  # Check if both are non-empty
        filtered_generated_summaries.append(pred)
        filtered_references.append(ref)

# Ensure the filtered lists are populated before calculating BLEU and ROUGE
print(f"Filtered Generated Summaries Count: {len(filtered_generated_summaries)}")
print(f"Filtered References Count: {len(filtered_references)}")

# BLEU score calculation
def calculate_bleu(filtered_generated_summaries, filtered_references, tokenizer):
    bleu_scores = []
    for pred, ref in zip(filtered_generated_summaries, filtered_references):
        pred_tokens = tokenizer.tokenize(pred)
        ref_tokens = tokenizer.tokenize(ref)
        if pred_tokens and ref_tokens:  # Ensure both are non-empty
            score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothing_function)
            bleu_scores.append(score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    return avg_bleu_score

# Calculate BLEU
bleu_score = calculate_bleu(filtered_generated_summaries, filtered_references, tokenizer)
print(f"BLEU Score: {bleu_score}")



/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Filtered Generated Summaries Count: 6847
Filtered References Count: 6847
BLEU Score: 0.05685980408115119


In [26]:
from rouge_score import rouge_scorer
from unidecode import unidecode

# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
# Define a function to preprocess and tokenize Bengali text
def preprocess_text(text):
    text = unidecode(text)
    tokens = text.split()
    return ' '.join(tokens)

# Create lists to store individual scores
rouge1_f1_scores = []
rouge1_precision_scores = []
rouge1_recall_scores = []
rouge2_f1_scores = []
rouge2_precision_scores = []
rouge2_recall_scores = []
rougeL_f1_scores = []
rougeL_precision_scores = []
rougeL_recall_scores = []

for ref, pred in zip(references, generated_summaries):
    candidate = preprocess_text(pred)
    reference = preprocess_text(' '.join(ref))
    scores = scorer.score(reference, candidate)

    rouge1_f1_scores.append(scores['rouge1'].fmeasure)
    rouge1_precision_scores.append(scores['rouge1'].precision)
    rouge1_recall_scores.append(scores['rouge1'].recall)
    rouge2_f1_scores.append(scores['rouge2'].fmeasure)
    rouge2_precision_scores.append(scores['rouge2'].precision)
    rouge2_recall_scores.append(scores['rouge2'].recall)
    rougeL_f1_scores.append(scores['rougeL'].fmeasure)
    rougeL_precision_scores.append(scores['rougeL'].precision)
    rougeL_recall_scores.append(scores['rougeL'].recall)

# Calculate the average scores
avg_rouge1_f1 = sum(rouge1_f1_scores) / len(rouge1_f1_scores)
avg_rouge1_precision = sum(rouge1_precision_scores) / len(rouge1_precision_scores)
avg_rouge1_recall = sum(rouge1_recall_scores) / len(rouge1_recall_scores)
avg_rouge2_f1 = sum(rouge2_f1_scores) / len(rouge2_f1_scores)
avg_rouge2_precision = sum(rouge2_precision_scores) / len(rouge2_precision_scores)
avg_rouge2_recall = sum(rouge2_recall_scores) / len(rouge2_recall_scores)
avg_rougeL_f1 = sum(rougeL_f1_scores) / len(rougeL_f1_scores)
avg_rougeL_precision = sum(rougeL_precision_scores) / len(rougeL_precision_scores)
avg_rougeL_recall = sum(rougeL_recall_scores) / len(rougeL_recall_scores)

In [27]:
# Print the average scores
print("Average Rouge-1 F1 Score:", avg_rouge1_f1)
print("Average Rouge-1 Precision:", avg_rouge1_precision)
print("Average Rouge-1 Recall:", avg_rouge1_recall)

print("Average Rouge-2 F1 Score:", avg_rouge2_f1)
print("Average Rouge-2 Precision:", avg_rouge2_precision)
print("Average Rouge-2 Recall:", avg_rouge2_recall)

print("Average Rouge-L F1 Score:", avg_rougeL_f1)
print("Average Rouge-L Precision:", avg_rougeL_precision)
print("Average Rouge-L Recall:", avg_rougeL_recall)

Average Rouge-1 F1 Score: 0.007826638760173693
Average Rouge-1 Precision: 0.0764327444296773
Average Rouge-1 Recall: 0.004157417230515951
Average Rouge-2 F1 Score: 0.0003857630018084146
Average Rouge-2 Precision: 0.003921362216966129
Average Rouge-2 Recall: 0.00020427769534275207
Average Rouge-L F1 Score: 0.007826638760173693
Average Rouge-L Precision: 0.0764327444296773
Average Rouge-L Recall: 0.004157417230515951


In [ ]:
generated_summaries

In [28]:
# from torch.utils.data import DataLoader
# import torch

# # Assuming tokenized_df_test has been defined and is in the correct format
# # Create DataLoader for the test set
# sample_dataloader = DataLoader(
#     tokenized_df_test,  # use the tokenized version
#     collate_fn=data_collator,  # Ensure this collator is defined to handle padding
#     batch_size=2
# )

# predictions = []
# references = []

# # Process the entire test set
# for batch in sample_dataloader:
#     with torch.no_grad():
#         # Ensure input_ids and labels are moved to the correct device (e.g., GPU)
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         # Generate predictions
#         preds = model.generate(
#             input_ids,
#             num_beams=2,
#             num_return_sequences=1,
#             no_repeat_ngram_size=1,
#             remove_invalid_values=True,
#             max_length=128
#         )
    
#     # Decode the predictions and labels
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Store the decoded predictions and labels
#     predictions.extend(decoded_preds)
#     references.extend(decoded_labels)

# # Prepare the references for metric evaluation (references need to be wrapped in lists)
# references = [[ref] for ref in references]

# # Compute the metrics using already decoded text (BLEU/ROUGE)
# metrics = metrics_func(predictions=predictions, references=references)
# print(metrics)


In [29]:
# def metrics_func(eval_arg):
#     preds, labels = eval_arg

#     # Ensure the Bengali punctuation logic is applied
#     text_preds = [(p if p.endswith(("!", "?", "।")) else p + "।") for p in preds]
#     text_labels = [(l if l.endswith(("!", "?", "।")) else l + "।") for l in labels]

#     # Tokenize sentences in Bengali
#     sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
#     text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
#     text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

#     # Compute the ROUGE score directly from the text
#     return rouge_metric.compute(predictions=text_preds, references=text_labels)


In [30]:
# from datasets import load_metric

# # Load ROUGE and/or BLEU metrics
# rouge = load_metric("rouge", trust_remote_code=True)
# bleu = load_metric("bleu")

# def metrics_func(predictions, references):
#     rouge_result = rouge.compute(predictions=predictions, references=references)
#     bleu_result = bleu.compute(predictions=predictions, references=references)
#     return {"rouge": rouge_result, "bleu": bleu_result}


In [31]:
# # Compute the metrics using already decoded text (BLEU/ROUGE)
# metrics = metrics_func(predictions=predictions, references=references)
# print(metrics)